In [7]:
# Load in packages
from pandas_datareader import data as DataReader
from datetime import datetime
import numpy as np
import pandas as p
from itertools import chain
import matplotlib.pyplot as plt
import operator
import sys

%matplotlib inline

In [8]:
################################################################################
# CHOOSE WHICH TRAIN DATASET TO USE
################################################################################
train = np.array(p.read_table('C:/Users/Aiqing-Jiang/Desktop/stock_market_prediction-master/training.csv', sep = ","))


In [9]:
################################################################################
# READ IN THE KAGGLE TRAIN DATA
################################################################################
"""
Function
--------
normalize10day

Replaces all min/max/vol data with 0, and divides all stock data\
by the opening price on the first day

Inputs
-------
stocks : (Nexample, Nfeature) array
        data for stocks, with 10 days of OPEN, MIN, MAX, CLOSE, VOL respectively
"""   
def normalize10day(stocks):
    def process_column(i):
        if operator.mod(i, 5) == 1:
            return stocks[:,i] * 0
        if operator.mod(i, 5) == 2:
            return stocks[:,i] * 0
        if operator.mod(i, 5) == 4:
            return stocks[:,i] * 0
            #return np.log(stocks[:,i] + 1)
        else:
            return stocks[:,i] / stocks[:,0]
    n = stocks.shape[0]
    stocks_dat =  np.array([ process_column(i) for i in range(46)]).transpose()
    return stocks_dat

if train.shape[0] == 94: # if we're using the kaggle training data
    n_windows = 490
    windows = range(n_windows)
    
    # we convert the 500 day data into a stack of 10 day data.
    X_windows = [train[:,range(1 + 5*w, 47 + 5*w)] for w in windows]
    X_windows_normalized = [normalize10day(w) for w in X_windows]
    
    X = np.vstack(X_windows_normalized)
    
    # read in the response variable and convert to indicators
    y_stockdata = np.vstack([train[:, [46 + 5*w, 49 + 5*w]] for w in windows])
    y = (y_stockdata[:,1] > y_stockdata[:,0]) + 0
    
################################################################################
# READ IN THE MODERN TRAIN DATA
################################################################################

else:
    # chain.from_iterable is basically a "flatten" function, that takes a list of lists and 
    # converts it to one list
    # columns we want are just the opening and closing prices
    columns_we_want = list(chain.from_iterable([[5 * x, 5 * x + 3] for x in range(10)]))[:-1]
    
    # we get our matrix of open and close prices, and normalize the data such that all data
    # is divided by the opening price on the first day
    X = np.array([l/l[0] for l in train[:, columns_we_want]])
    
    # we make indicators of whether or not the stock went up that day.
    y = (train[:, 48] > train[:, 45]) + 0

